In [18]:
from tkinter import *
import time
import pprint
import random
import nbimporter
from ia import Ia_dumb
Ia_dumb()

running


<h2><b>GAME</b></h2>

In [19]:
#DRAWING FUNC
def pack_util(p) : 
    return p.pack()

In [49]:
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True):
        #INIT
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.direction = ""
        self.commands_history = [self.direction]
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        if play_btn : 
            #PLAY BUTTON
            self.play_button = Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2 )
            pack_util(self.play_button)
            
    def create_canvas(self) :
        canvas = Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :
        #create dict with slot of rows
        pos_dict = {x : [] for x in range(0, self.game_size + 1, self.pixel_size)}
        
        #fill with all points possible in a row 
        for x in range(0, self.game_size + 1) :
            for y in range(0, self.game_size + 1) :
                if (x == 0 and y == 0) or (x % self.pixel_size == 0 and y % self.pixel_size == 0):
                    pos_dict[x].append((x, y))
                    
        #create matrix with couple of couple of coord by rows
        count = -1       
        for x in pos_dict:
            for i in range(len(pos_dict[x])):
                if x != self.game_size :
                    if count == int(self.game_size / self.pixel_size)-1 :
                        count = -1
                    else : 
                        count += 1
                        
                    if i < int(self.game_size / self.pixel_size) :
                        self.grid_pos[count].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])                
        
        
    def set_key_event(self, event: Event, canvas) :
        if event.char in self.commands :
            print(event.char, self.commands[event.char])
            self.direction = self.commands[event.char]
            if self.commands_history[len(self.commands_history) - 2] != self.direction :
                print(self.commands_history)
                self.commands_history.append(self.direction)
                self.snake.move(self.direction, canvas, self.commands_history)
            
        
    def play(self) :
        print("Playing")
        #REMOVE PLAY BUTTON
        self.play_button.destroy()
        self.canvas = self.create_canvas()
        print(self.canvas)
        if len(self.tinker.winfo_children()) == 1 :
            
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            self.snake = Snake(self.tinker, self.game_size, False)
            self.snake.create_snake(self.canvas)
            
            self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
            
            pack_util(self.canvas)

In [53]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        print("apple", canvas)
        self.apple = canvas.create_rectangle(
                self.grid_pos[self.init_pos][self.init_pos][0][1],
                self.grid_pos[self.init_pos][self.init_pos][0][0],
                self.grid_pos[self.init_pos][self.init_pos][1][1],
                self.grid_pos[self.init_pos][self.init_pos][1][0],
                outline="#000000",
                fill="#ff8f26"
            )

In [56]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.snake_positions = [(5,5), (6,5), (7, 5)]
        self.direction = "none"
        
    def create_snake(self, canvas):
        print("snake", canvas)
        for x_position, y_position in self.snake_positions :
            self.snake_part = canvas.create_rectangle(
                self.grid_pos[x_position][y_position][0][1],
                self.grid_pos[x_position][y_position][0][0],
                self.grid_pos[x_position][y_position][1][1],
                self.grid_pos[x_position][y_position][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake')
            )
            
    def move(self, direction, canvas, commands_history) :
        movement = True
        direction_is_diff = False
        
        while movement : 
            for i, snake_id in enumerate(canvas.find_withtag("snake")):
                coords = canvas.coords(snake_id)
                draw = False
                #print(coords)

                x = self.snake_positions[i][0]
                y = self.snake_positions[i][1]
                
                all_x = [j[0] for j in self.snake_positions]
                all_y = [k[1] for k in self.snake_positions]
                
                max_x = max(all_x)
                max_y = max(all_y)
                
                if direction is "right" :
                    y = max_y
                    if x < len(self.grid_pos[x]) - 1:
                        if direction_is_diff:
                            x += i+1
                            if i is len(canvas.find_withtag("snake")) - 1 :
                                direction_is_diff = False
                                print("changed is diff", direction_is_diff)
                        else :
                            x += 1
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                
                if direction is "left" :
                    y = max_y
                    if x < len(self.grid_pos[x]) - 1:
                        if x > 0 :
                            if direction_is_diff :
                                x -= i+1
                                if i is len(canvas.find_withtag("snake")) - 1 :
                                    direction_is_diff = False
                                    print("changed is diff", direction_is_diff)
                            else :
                                x -= 1

                            self.snake_positions[i] = (x,y)
                            draw = True
                        else :
                            movement = False
                            break
                    else :
                        movement = False
                        break
                        
                if direction is "bottom" :
                    if y < len(self.grid_pos) - 1:
                        x = self.snake_positions[0][0]
                        if direction_is_diff is False:
                            y += i+1
                            if i is len(canvas.find_withtag("snake")) - 1 :
                                direction_is_diff = True
                                print("changed is diff", direction_is_diff)
                        else :
                            y += 1
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                print(f"Snake_part-{i} : {x, y}, direction : {direction}, direction_is_diff : {direction_is_diff}")
                if draw : 
                    canvas.coords(
                        snake_id,
                        self.grid_pos[x][y][0][1],
                        self.grid_pos[x][y][0][0],
                        self.grid_pos[x][y][1][1],
                        self.grid_pos[x][y][1][0],
                    )
            canvas.update()
            time.sleep(0.1)
        print(direction)

In [57]:
root = Tk()
snake_game = Snake_game(root)
root.mainloop()

Playing
.!canvas
apple .!canvas
snake .!canvas
d right
['']
Snake_part-0 : (6, 5), direction : right, direction_is_diff : False
Snake_part-1 : (7, 5), direction : right, direction_is_diff : False
Snake_part-2 : (8, 5), direction : right, direction_is_diff : False
Snake_part-0 : (7, 5), direction : right, direction_is_diff : False
Snake_part-1 : (8, 5), direction : right, direction_is_diff : False
Snake_part-2 : (9, 5), direction : right, direction_is_diff : False
Snake_part-0 : (8, 5), direction : right, direction_is_diff : False
Snake_part-1 : (9, 5), direction : right, direction_is_diff : False
Snake_part-2 : (10, 5), direction : right, direction_is_diff : False
Snake_part-0 : (9, 5), direction : right, direction_is_diff : False
Snake_part-1 : (10, 5), direction : right, direction_is_diff : False
Snake_part-2 : (11, 5), direction : right, direction_is_diff : False
Snake_part-0 : (10, 5), direction : right, direction_is_diff : False
Snake_part-1 : (11, 5), direction : right, direction

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-49-e6859fbf2a70>", line 78, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-49-e6859fbf2a70>", line 61, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-56-a639e37aeb7d>", line 25, in move
    for i, snake_id in enumerate(canvas.find_withtag("snake")):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 2548, in find_withtag
    return self.find('withtag', tagOrId)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 2522, in find
    self.tk.call((self._w, 'find') + args)) or ()
_tkinter.TclError: invalid command name ".!canvas"
Exce